In [1]:
import os
import sys
from pathlib import Path # if you haven't already done so
file = Path(os.path.dirname(os.path.abspath("__file__"))).resolve()
parent, root = file.parent, file.parents[1]
sys.path.append(str(root))

import torch
from dtmgp.utils.sparse_grid.hyperbolic_cross import HyperbolicCrossDesign
from dtmgp.utils.sparse_grid.sgdesign import SparseGridDesign
from dtmgp.kernels.laplace_kernel import LaplaceProductKernel
from dtmgp.utils.sparse_weights.chol_inv import mk_chol_inv, tmk_chol_inv

# Cholesky inverse in one dimension

In [27]:
design_class = HyperbolicCrossDesign
deg = 2
input_bd = [0,1]

dyadic_design = design_class(dyadic_sort=True, indices_sort=True)(deg=deg, input_bd=input_bd)
markov_kernel = LaplaceProductKernel(lengthscale=None)

Rinv = mk_chol_inv(dyadic_design, markov_kernel = markov_kernel, upper = True)

In [28]:
print(dyadic_design.points)
print(dyadic_design.sorted_vals_indices)
print(dyadic_design.indices_sort)

tensor([0.5000, 0.2500, 0.7500])
(tensor([0.2500, 0.5000, 0.7500]), tensor([1, 0, 2]))
tensor([1, 0, 2])


In [29]:
ker_input = dyadic_design.points
K_true = markov_kernel(ker_input, ker_input)
#print(K_true[:5,:5])

R_true = torch.cholesky(K_true, upper=True)
Rinv_true = torch.linalg.inv(R_true)
print(Rinv_true[:5,:5])

Kinv_true = torch.linalg.inv(K_true)
#print(Kinv_true[:5,:5])

tensor([[ 1.0000, -1.2416, -1.2416],
        [ 0.0000,  1.5942,  0.0000],
        [ 0.0000,  0.0000,  1.5942]])


In [30]:
Rinv_dense = Rinv.to_dense()
print(Rinv_dense[:15,:15])

Kinv_sp = ( Rinv @ Rinv.T ).to_dense()
#print(Kinv_sp[:5,:5])

tensor([[ 2.0206, -1.2416, -1.2416],
        [-0.9795,  1.5942,  0.0000],
        [-0.9795,  0.0000,  1.5942]])


In [12]:
abs(Kinv_sp - Kinv_true)

tensor([[3.0830, 1.9793, 1.9793],
        [1.9793, 0.9595, 0.9595],
        [1.9793, 0.9595, 0.9595]])

In [7]:
torch.allclose(Kinv_sp.to_dense(), Kinv_true)

False

# Cholesky inverse for sparse grids

In [31]:
# initial setting
d = 2 # dimension
eta = 3 # level
input_bd = [[0,1]]*d # None
design_class = HyperbolicCrossDesign
dyadic_sort = True
indices_sort = True

# generate sparse grid design
sg = SparseGridDesign(d, eta, input_bd=input_bd, design_class=design_class).gen_sg(dyadic_sort=dyadic_sort, indices_sort=indices_sort)
sg.pts_set

tensor([[0.2500, 0.5000],
        [0.5000, 0.2500],
        [0.5000, 0.5000],
        [0.5000, 0.7500],
        [0.7500, 0.5000]])

In [40]:
tensor_markov_kernel = LaplaceProductKernel(lengthscale=1.)
Rinv = tmk_chol_inv(sparse_grid_design=sg, 
                    tensor_markov_kernel=tensor_markov_kernel, 
                    upper = True)
Rinv.to_dense()

tensor([[ 1.2578,  0.0000, -0.4846,  0.0000,  0.0000],
        [ 0.0000,  1.2578, -0.4846,  0.0000,  0.0000],
        [-0.7629, -0.7629,  1.5095, -0.7629, -0.7629],
        [ 0.0000,  0.0000, -0.4846,  1.2578,  0.0000],
        [ 0.0000,  0.0000, -0.4846,  0.0000,  1.2578]])

In [47]:
#ker_input = torch.tensor([[0.5,0.5],[0.5,0.25],[0.5,0.75],[0.25,0.5],[0.75,0.5]])
ker_input = sg.pts_set
K_true = tensor_markov_kernel(ker_input, ker_input)
Kinv_true = torch.linalg.inv(K_true)
# print(K_true)
# print(Kinv_true)

R_true = torch.cholesky(K_true, upper=True)
Rinv_true = torch.linalg.inv(R_true)
print(Rinv_true)

tensor([[ 1.0000e+00, -7.6287e-01, -9.7955e-01, -9.3188e-08, -9.3188e-08],
        [ 0.0000e+00,  1.2578e+00, -9.7955e-01, -1.5307e-07, -1.5307e-07],
        [ 0.0000e+00,  0.0000e+00,  2.0206e+00, -1.2416e+00, -1.2416e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.5942e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.5942e+00]])
